In [5]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEy is not set ion the environment variables.")

In [ ]:
%pip install openai-agents python-dotenv -qU

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from agents import Agent, Runner

agent = Agent(
    name="Ted Agent",
    instructions="You are a helpful assistant. Respond on in all caps.",
    model="gpt-4.1-nano"
)

result = await Runner.run(agent, "Hello, how are you?")
result.final_output

"HELLO! I'M DOING WELL, THANK YOU. HOW CAN I HELP YOU TODAY?"

In [ ]:
from pydantic import BaseModel

class Recipe(BaseModel):
    title: str
    ingredients: list[str]
    instrunctions: str
    cooking_time: int
    servings: int

In [ ]:
recipe_agent = Agent(
    name="Recipe Agent",
    model="gpt-4.1-nano",
    instructions="your are an expert chef for creating recipes. You will be given the name of a food and your job "
    " is to create that as an actual detailed recepie. The cooking time should be in minutes.",
    output_type=Recipe
)

response = await Runner.run(recipe_agent, "Italian pizza")
recipe = response.final_output

Recipe(title='Italian Pizza', ingredients=['2 1/4 cups all-purpose flour', '1 packet (2 1/4 teaspoons) active dry yeast', '1 teaspoon sugar', '1 teaspoon salt', '3/4 cup warm water (110°F)', '2 tablespoons olive oil', '1/2 cup pizza sauce', '2 cups shredded mozzarella cheese', 'Toppings of choice (pepperoni, vegetables, etc.)'], instrunctions='In a small bowl, dissolve yeast and sugar in warm water and let sit for 5 minutes until frothy. In a large bowl, combine flour and salt. Add the yeast mixture and olive oil to the flour mixture and knead until smooth and elastic, about 8-10 minutes. Cover the dough and let rise in a warm place for about 1 hour, or until doubled in size. Preheat oven to 475°F (245°C). Punch down the dough and roll it out on a floured surface to your desired thickness. Transfer to a pizza baking sheet or stone. Spread pizza sauce over the dough, then top with shredded mozzarella and your chosen toppings. Bake in the preheated oven for 12-15 minutes, or until the cr

## Tool Calling (Python code)

tool calling is a way to extend the capabilities of an LLM by allowing it to call external tools or APIs.
this can be useful for tasks that require access to external resources.

In [4]:
from json import tool
from agents import Agent, Runner, function_tool

@ function_tool
def get_weather(city: str) -> str:
    print(f"Getting weather for {city}")
    return "synny"


@function_tool
def get_temperature(city: str) -> str:
    print(f"Getting remperature for {city}")
    return "70 degrees"


weather_agent = Agent(
    name="Weather Agent",
    model="gpt-4.1-nano",
    instructions="you are a weather forecaster. You are given a city and you need to tell the weather and temperature for that city.",
    tools=[get_weather, get_temperature]
)


result = await Runner.run(weather_agent, "What is the weather and temperature in Seoul?")
result.final_output

Getting weather for Seoul
Getting remperature for Seoul


'The weather in Seoul is sunny, and the temperature is 70 degrees.'

## built-in function
WebSerchTool, cost according to context size for searching

<info> duck.go, firecrawl

In [7]:
from agents import WebSearchTool

news_agent = Agent(
    name="News Agent",
    model="gpt-4o-mini",
    instructions="You are a news agent. You will be given a topic and you need to search the web for the latest news about Southe Korea politics.",
    tools=[WebSearchTool()],
)

result = await Runner.run(news_agent, "find news")
print(result.final_output)

Here is a summary of recent developments in South Korean politics:

**Stock Market Reforms and Economic Measures**

- **Corporate Governance Reforms**: President Lee Jae-myung has initiated significant reforms aimed at improving corporate governance and addressing the "Korea Discount," where South Korean companies are undervalued due to weak shareholder protections. Proposed measures include imposing fiduciary duties to shareholders, enhancing minority shareholder rights, and cracking down on treasury stock practices. These reforms aim to boost the domestic stock market's appeal and align valuations with emerging market or Japanese standards. ([ft.com](https://www.ft.com/content/5ee12c61-b561-4c36-9c16-77a397adda4f?utm_source=openai))

- **Tax Incentives for Dividends**: To encourage higher dividend payouts, President Lee announced plans to reform the country's tax system. The proposed changes aim to make the domestic stock market more attractive to investors. ([reuters.com](https://ww

## Agent with Input

In [9]:
from dotenv import load_dotenv
import os
from agents import Agent, Runner, WebSearchTool

load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables.")


ted_agent = Agent(
    name="Websearch Agent",
    model="gpt-4o-mini",
    instructions="you a web search agent, you will be given a topic and you need to search the web for the latest news about that topic." +
    "Compile the information you find into a concise 1 paragraph summary. No markdown, just plain text.",
    tools=[WebSearchTool()],
)
while True:
    query = input("Enter a search query (or 'quit' to exit): ")
    if query.lower() == "quit":
        break

    result = await Runner.run(ted_agent, input=query)
    print(f"\nResult:")
    print(result.final_output)
    print("\n" + "=" * 50 + "\n")


Result:
Hello! How can I assist you today?





## Handoff

In [ ]:
from pydantic import BaseModel


class Tutorial(BaseModel):
    outline: str
    tutorial: str

tutorial_generator = Agent(
    name="Tutorial Generator",
    handoff_description="An outline for a programming tutorial",
    model="gpt-4.1-nano",
    instructions=("Given a programming topic and an outline, your job is to generate code snippets for each secton of the outline. Format the tutorial in Markdown using a mix of text for explanation and code snippets for examples. Where it makes sense, include comments in teh code snippets to further explain the code."),
    output_type=Tutorial
)

outline_builder = Agent(
    name="Outline Builder",
    model="gpt-4.1-nano",
    instructions="Given a particular programming topic, your job is to help come up with a tutorial. You will do that by crating an outline.After making the outline, hand it to the totorial generator agent.",
    handoffs=[tutorial_generator],
)

tutorial_response = await Runner.run(outline_builder, "Loops to build list of integer with single line of code in Python")
print(tutorial_response.final_output)


outline='1. Using list comprehensions\n2. Using the `map` function\n3. Using a `for` loop with list addition in a single line' tutorial='## Building a list of integers with a single line in Python\n\nPython provides elegant and concise ways to create lists of integers using single line expressions.\n\n### 1. Using list comprehensions\n\nList comprehensions are a compact way to process and create lists in Python.\n\n```python\n# Create a list of integers from 0 to 9\nnumbers = [i for i in range(10)]\nprint(numbers)  # Output: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]\n```\n\n### 2. Using the `map` function\n\nThe `map` function applies a given function to all items in an iterable.\n\n```python\n# Create list of integers from 0 to 9 using map\nnumbers = list(map(int, range(10)))\nprint(numbers)  # Output: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]\n```\n\n### 3. Using a `for` loop with list addition in one line\n\nYou can also execute a `for` loop in a single line using the `[]` syntax.\n\n```python\n# Create 

## Agent use case1 (Root Agent and Child Agents)

In [22]:
from agents import Agent, Runner, handoff, RunContextWrapper

history_tutor_agent = Agent(
    name="History Tutor",
    model="gpt-4.1-nano",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain import events and context clearly."
)

math_tutor_agent = Agent(
    name="Math Tutor",
    model="gpt-4.1-nano",
    handoff_description="Specialist agent for mathematical problems",
    instructions="You provide assistance with math queries. Explain your reasoning at each step and include examples."

)

def ted_math_handoff(ctx: RunContextWrapper[None]):
    print("Handing off to Math Tutor Agent")

def ted_history_handoff(ctx: RunContextWrapper[None]):
    print("Handing off to History Tutor Agent")

triage_agent = Agent(
    name="Trige Agent",
    model="gpt-4.1-nano",
    instructions="You determine which agent to hand off to based on the user's questions." +
    "If neither agent is suitable, provide a general response.",
    handoffs=[
        handoff(history_tutor_agent, on_handoff=ted_history_handoff),
        handoff( math_tutor_agent, on_handoff=ted_math_handoff)
    ]

)



In [26]:
result = await Runner.run(triage_agent, "How do I add two numbers(2, 5) ?")
print(result.final_output)

Handing off to Math Tutor Agent
To add two numbers, you simply combine their values. 

Let's add 2 and 5:

2 + 5 = 7

So, the sum of 2 and 5 is **7**.


In [27]:
result = await Runner.run(triage_agent, "How did World War2 start? Explain the key events leading up to it.")
print(result.final_output)

Handing off to History Tutor Agent
World War II began as a result of a series of complex and interrelated events leading up to 1939. Here are some of the key events and factors that contributed to its outbreak:

1. Treaty of Versailles (1919): The punitive peace treaty ending World War I imposed heavy reparations and territorial losses on Germany. This created economic hardship and national resentment, fueling nationalist sentiments.

2. Rise of Fascism and Militarism: In the 1930s, fascist regimes rose to power in Germany under Adolf Hitler, Italy under Benito Mussolini, and militarist factions in Japan. These regimes pursued aggressive expansionist policies.

3. Germany’s Rearmament and Anschluss: Germany violated the Treaty of Versailles by rearming and unifying with Austria in 1938 (Anschluss), demonstrating its willingness to defy international agreements.

4. Munich Agreement and Appeasement: In 1938, Britain and France adopted a policy of appeasement, allowing Germany to annex p

## Agent - use case2

In [ ]:
from agents import Agent, function_tool, RunContextWrapper, handoff


class ManagerEscalation(BaseModel):
    issue: str
    why: str

# this is an example of create a ticket with a ticketing system when the issues is escalated to a manager agent.
@function_tool
def create_ticket(issue: str):
    print(f"Creating a ticket for the customer issues: {issue}")
    return "Ticket created successfully."

manager_agent = Agent(
    name="Manager Agent",
    handoff_description="Handles escalated issues that require managerial attention",
    instructions=("""
        You handle escalated customer issues that the initial customer service agent could not resolve.
        You will receive the issue and the reason for escalation. If the issue cannot be immediately resolved for the customer, 
        Create a ticket in the system and inform the customer.
        """
    ),
    model="gpt-4.1-nano",
    tools=[create_ticket]
)

# This is a normal function not a tool
# Need to store somewhere like a database or a ticketing system
# issue and why are the fields that will be passed to the manager agent when the issue is escalated in a class called ManagerEscalation
def ted_manager_handoff(ctx: RunContextWrapper[None], input: ManagerEscalation):
    print("Escalating to manager agent: ", input.issue)
    print("Reason for escalation: \n\n", input.why)

customer_service_agent = Agent(
    name="Customer Service Agent",
    model="gpt-4.1-nano",
    instructions=("""
        You assist customers with general inquiries and basic troubleshooting. 
        If the issue is beyond your scope, escalate it to the Manager Agent along with the reason
        Why you can't resolve the issue.
    """),
    handoffs=[
        handoff(agent=manager_agent,input_type=ManagerEscalation,on_handoff=ted_manager_handoff)
    ]
)

In [35]:
result = await Runner.run(customer_service_agent, "I want a refund, but your system is now working. The website is down.")
print(result.final_output)

Escalating to manager agent:  Requesting a refund while the website is down.
Reason for escalation: 

 Unable to process the refund or verify the request due to website outage.
It appears that the website is currently down, which is preventing us from processing your refund at the moment. I will escalate this issue to our manager so they can assist you further.
